In [1]:
import pandas as pd
import numpy as np

# import the data 
csv_file = '../data/1/FF-FF-30-02-3E-5C.csv'
df = pd.read_csv(csv_file)

df.head()

,distance,rssi
0,1,-39
1,1,-39
2,1,-39
3,1,-45
4,1,-53


In [2]:
calc_dist = lambda A,B : ((A[0]-B[0])**2 + (A[1]-B[1])**2)**0.5

def get_real_rssi(distance:float):
  # get the closest distance to the given distance
  closest_distance = df['distance'].iloc[(df['distance']-distance).abs().argsort()[:1]].values[0]
  # get random rssi value for that distance
  rssi = df[df['distance'] == closest_distance]['rssi'].sample().values[0]
  return rssi

In [3]:
# p = [-1.53921836e-05,  4.65072912e-03, -5.29947347e-01,  2.87587266e+01,-7.46768727e+02,  7.45399589e+03]
p = [-3.29890901e-04,  9.73375320e-02, -9.58804523e+00,  3.88729903e+02,-5.55883921e+03]

get_distance = lambda rssi: sum([p[i]*abs(rssi)**(len(p)-i-1) for i in range(len(p))])

In [40]:
from ipycanvas import Canvas, hold_canvas
import random

scale = 0.5
WIDTH = 1521
HEIGHT = 748
radius = 15

canvas = Canvas(width=WIDTH, height=HEIGHT)

antennas = [
    dict(x=0, y=0, radius=radius),
    dict(x=0, y=HEIGHT, radius=radius),
    dict(x=900, y=HEIGHT, radius=radius),
    dict(x=900, y=0, radius=radius),
    dict(x=WIDTH, y=HEIGHT, radius=radius),
]

# with hold_canvas():
canvas.fill_style = 'black'
canvas.fill_rect(WIDTH-580, 0, WIDTH, 186)


# pick a random position for the beacon 
x = random.randint(0, WIDTH)
y = random.randint(0, HEIGHT)


real_distances = [
  calc_dist((x,y), (antenna['x'], antenna['y']))
  for antenna in antennas
]

real_rssi = [
    get_real_rssi(dist)
    for dist in real_distances
]

distances = [
  get_distance(rssi) for rssi in real_rssi
]

# print(*zip(real_distances, real_rssi, distances), sep='\n')

prevSquare = None


# draw the circles
for i, antenna in enumerate(antennas):
  canvas.stroke_style = 'blue'
  canvas.stroke_rect(antenna['x']-distances[i], antenna['y']-distances[i], distances[i]*2, distances[i]*2)
  
  tolerance = 3 * 100
  max_distance = max(0,distances[i] + tolerance)
  min_distance = max(0,distances[i] - tolerance)
  
  if distances[i] > 9_00 :
    continue
    
  # draw the rectangle with fill opacity 0.5
  canvas.fill_style = 'rgba(0, 0, 200, 0.1)'
  canvas.fill_rect(antenna['x']-max_distance, antenna['y']-max_distance, max_distance*2, max_distance*2)
  
  canvas.fill_style = 'rgba(255, 255, 255, 0.0)'
  canvas.fill_rect(antenna['x']-min_distance, antenna['y']-min_distance, min_distance*2, min_distance*2)
  
  # intersection with prevSquare
  A = (antenna['x']-max_distance, antenna['y']-max_distance)
  B = (antenna['x']+max_distance, antenna['y']-max_distance)
  C = (antenna['x']+max_distance, antenna['y']+max_distance)
  D = (antenna['x']-max_distance, antenna['y']+max_distance)
  if not prevSquare:
    prevSquare = [min(A[0],B[0],C[0],D[0]), min(A[1],B[1],C[1],D[1]), max(A[0],B[0],C[0],D[0]), max(A[1],B[1],C[1],D[1])]
  else:
    a,b,c,d = prevSquare
    prevSquare = [
      max(a, min(A[0],B[0],C[0],D[0])),
      max(b, min(A[1],B[1],C[1],D[1])),
      min(c, max(A[0],B[0],C[0],D[0])),
      min(d, max(A[1],B[1],C[1],D[1])),
    ]
    
    
  # print(prevSquare)
  

# # draw the predicted position
# canvas.fill_style = 'rgba(255, 0, 0, 0.5)'
# canvas.fill_rect(prevSquare[0], prevSquare[1], prevSquare[2], prevSquare[3])

# draw the beacon
canvas.fill_style = 'red'
canvas.fill_circle(x, y, 10)

# draw the antennas
for antenna in antennas: 
  canvas.fill_style = 'green'
  canvas.fill_circle(antenna['x'], antenna['y'], antenna['radius'])

canvas.scale(scale)
canvas

Canvas(height=748, width=1521)